In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
# --- Configuration ---
# IMPORTANT: Update this path to where your extracted FER-2013 dataset is located.
# For example, if you extracted fer2013.zip and the 'train' and 'test' folders
# are inside a folder named 'fer2013' at 'D:/Projects/Deep Learning Project/'
base_dataset_path = "D:/Projects/Deep Learning Project" # <--- **UPDATE THIS PATH TO YOUR FER2013 FOLDER**

IMG_HEIGHT = 48  # FER-2013 images are 48x48 pixels
IMG_WIDTH = 48
BATCH_SIZE = 32
NUM_CLASSES = 7  # Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral
EPOCHS = 100      # Number of training iterations. You can increase this for better performance.
SEED = 42        # For reproducibility of random operations

# Define emotion labels for consistent mapping and display
# FER2013 labels are usually: 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
# This dictionary maps the integer labels to human-readable emotion names.
emotion_labels = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral"
}

# --- 1. Load Data with tf.keras.utils.image_dataset_from_directory ---
# This function is great because it infers labels from folder names.

try:
    print(f"Loading training data from: {os.path.join(base_dataset_path, 'train')}")
    train_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(base_dataset_path, 'train'),
        labels='inferred',
        label_mode='int',
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        interpolation='nearest',
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=SEED,
        color_mode='grayscale' # <--- CRITICAL CHANGE: Load images as grayscale
    )

    print(f"Loading test data from: {os.path.join(base_dataset_path, 'test')}")
    test_ds = tf.keras.utils.image_dataset_from_directory(
        os.path.join(base_dataset_path, 'test'),
        labels='inferred',
        label_mode='int',
        image_size=(IMG_HEIGHT, IMG_WIDTH),
        interpolation='nearest',
        batch_size=BATCH_SIZE,
        shuffle=False,
        seed=SEED,
        color_mode='grayscale' # <--- CRITICAL CHANGE: Load images as grayscale
    )

    # Get the actual class names detected by the loader (these will be the folder names)
    class_names_detected = train_ds.class_names
    print(f"\nDetected class names (from folder names): {class_names_detected}")

    # Preprocessing function: Rescale pixel values from [0,255] to [0,1]
    def preprocess(image, label):
        image = tf.cast(image, tf.float32) / 255.0
        return image, label

    # Apply preprocessing and optimize dataset loading for performance
    train_ds = train_ds.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    test_ds = test_ds.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    print(f"\nNumber of training batches: {len(train_ds)}")
    print(f"Number of test batches: {len(test_ds)}")

except Exception as e:
    print(f"\nError loading dataset: {e}")
    print(f"Please ensure the dataset is extracted correctly and '{os.path.join(base_dataset_path, 'train')}' and '{os.path.join(base_dataset_path, 'test')}' exist and contain emotion subfolders.")
    print("Exiting as data loading failed.")
    exit() # Exit if data loading fails

# --- 2. Build the Deep Learning Model (Simple CNN Architecture) ---
model = keras.Sequential([
    # Input layer: expects images of IMG_HEIGHT x IMG_WIDTH with 1 color channel (grayscale)
    # CRITICAL CHANGE: input_shape last dimension is 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
    layers.BatchNormalization(), # Helps stabilize training and speed up convergence
    layers.MaxPooling2D((2, 2)), # Reduces spatial dimensions
    layers.Dropout(0.25),        # Randomly sets a fraction of input units to 0 at each update during training time, which helps prevent overfitting.

    # Convolutional Block 2
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Convolutional Block 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    # Flatten the 3D output of convolutional layers to 1D for Dense layers
    layers.Flatten(),

    # Dense (Fully Connected) Layers
    layers.Dense(256, activation='relu'), # You had 256 here previously, keeping it.
    layers.BatchNormalization(),
    layers.Dropout(0.5), # Higher dropout for the dense layers

    # Output Layer: NUM_CLASSES neurons, 'softmax' activation for multi-class probabilities
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Display model architecture summary
print("\n--- Model Architecture Summary ---")
model.summary()


Loading training data from: D:/Projects/Deep Learning Project\train
Found 28709 files belonging to 7 classes.
Loading test data from: D:/Projects/Deep Learning Project\test
Found 7178 files belonging to 7 classes.

Detected class names (from folder names): ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

Number of training batches: 898
Number of test batches: 225

--- Model Architecture Summary ---


C:\Users\miyee\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 21, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 574,695 (2.19 MB)

 Trainable params: 573,799 (2.19 MB)

 Non-trainable params: 896 (3.50 KB)

In [4]:
# --- 3. Compile the Model ---
# Configure the model for training
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001), # Adam is a popular and effective optimizer
    loss=keras.losses.SparseCategoricalCrossentropy(),    # Suitable for integer labels (0, 1, 2, ...)
    metrics=['accuracy']                                  # Metric to monitor during training
)

In [5]:


# --- 4. Train the Model ---
print("\n--- Model Training ---")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds, # Using the test set as validation data for simplicity in this example
    verbose=1                # Display training progress
)

print("\n--- Training Complete ---")



--- Model Training ---
Epoch 1/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.2755 - loss: 2.2158 - val_accuracy: 0.4291 - val_loss: 1.4903
Epoch 2/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.4250 - loss: 1.5018 - val_accuracy: 0.4809 - val_loss: 1.3767
Epoch 3/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.4623 - loss: 1.4046 - val_accuracy: 0.4600 - val_loss: 1.3831
Epoch 4/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.4886 - loss: 1.3449 - val_accuracy: 0.4911 - val_loss: 1.3094
Epoch 5/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.5108 - loss: 1.2968 - val_accuracy: 0.5093 - val_loss: 1.3028
Epoch 6/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.5180 - loss: 1.2705 - val_accuracy: 0.5295 - val_loss: 1.2475
Epoch 7/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - accuracy: 0.5296 - loss: 1.2431 - val_accuracy: 0.5334 - val_loss: 1.2428
Epoch 8/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 16s 18ms/step - ac

In [6]:
# --- 5. Evaluate the Model on the Test Set ---
print("\n--- Model Evaluation ---")
test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


--- Model Evaluation ---
Test Loss: 1.4254
Test Accuracy: 0.5539


In [8]:
import pandas as pd
from datetime import date, timedelta
import datetime
# Save training history to versioned CSV in 'history' folder
history_df = pd.DataFrame(history.history)
history_df['time'] = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
os.makedirs('history', exist_ok=True)
history_filename = os.path.join('history', f"training_history_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
history_df.to_csv(history_filename, index=False)
print(f"📁 Training history saved to {history_filename}")

📁 Training history saved to history\training_history_20250614_003851.csv


In [9]:
final_train_accuracy = history.history['accuracy'][-1]
final_val_accuracy = history.history['val_accuracy'][-1]

print(f"\nFinal Training Accuracy (last epoch): {final_train_accuracy:.4f}")
print(f"Final Validation Accuracy (last epoch): {final_val_accuracy:.4f}")


Final Training Accuracy (last epoch): 0.7646
Final Validation Accuracy (last epoch): 0.5539


In [10]:
# Save Model and Label Map
import json

model.save('emotion_recognition_model.h5')
with open('emotion_labels.json', 'w') as f:
    json.dump(emotion_labels, f)

In [6]:

import pandas as pd
all_train_labels = []
for images, labels in train_ds:
    all_train_labels.extend(labels.numpy())

# Convert the list of labels to a Pandas Series for easy counting
train_labels_series = pd.Series(all_train_labels)

# --- Step 2: Get the counts for each emotion class ---
# .value_counts() gives the frequency of each unique label
# .sort_index() ensures the emotions are ordered by their numerical label (0, 1, 2, ...)
class_counts = train_labels_series.value_counts().sort_index()
print(class_counts)

0    3995
1     436
2    4097
3    7215
4    4965
5    4830
6    3171
Name: count, dtype: int64


In [8]:
ordered_emotion_names = [emotion_labels[i] for i in sorted(emotion_labels.keys())]
print(ordered_emotion_names)

['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
